In [19]:
import flywheel
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, 
    level=logging.INFO, 
    format='%(levelname)s: %(message)s')

logger = logging.getLogger('fw_remove_metadata')

logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s: %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)


logger.info("fw_remove_metadata starting up")


INFO: fw_remove_metadata starting up


In [20]:
# Specify project (required)
project_label = "22Q_812481"

# Specify sessions (optional)
session_labels = ["008120","008144"]

# Which file types to change metadata
allowed_ftypes = ["nifti"]

# Specify list of fields to delete
fields_for_deletion = [
    "AcquisitionDateTime",
    "AcqusitionTime",
    "PatientSex", 
    "SeriesInstanceUID",
    "DeviceSerialNumber", 
    "AcquisitionTime",
    "ReferringPhysicianName",
    "StationName",
    "InstitutionAddress",
    "InstitutionName",
    "InstitutionalDepartmentName",
    ###### ASL specific: ######
    "AcquisitionDuration",
    "ArterialLabelingType",
    "AverageB1LabelingPulses",
    "AverageLabelingGradient",
    "BolusCutOffDelayTime",
    "BolusCutOffFlag",
    "BolusCutOffTechnique",
    "BolusCutOffTimingSequence",
    "LabelingOrientation",
    "LookLocker",
    "M0",
    "PASLType",
    "PulseDuration",
    "PulseSequenceDetails"
    ]

In [ ]:
# Get client
fw = flywheel.Client()
assert fw, "Your Flywheel CLI credentials aren't set!"

# Get project object
project = fw.projects.find_first(            
    'label="{}"'.format(project_label))
assert project, "Project not found!"
logger.debug('Found project: %s (%s)',
                     project['label'], project.id)


In [14]:
# Get all sessions in project
sessions = project.sessions()

# Filter by optionally-specified session label list
if session_labels:
    sessions = [s for s in sessions if s.label in session_labels]

assert sessions, "No sessions found!"
logger.info(f"Metadata will be removed from {len(sessions)} sessions".)

INFO: Processing 2 sessions


In [18]:
# Get acquisitions in each session
i=0
for sess in sessions:
    i+=1
    #sess = sess.reload()
    acqs = sess.acquisitions()
    logger.info(f"Processing session: {sess.label} ({i}/{len(sessions)})")

    # Get files in each acquisition
    for acq in acqs:
        acq = acq.reload() # Can't load custom metadata without this line!!!
        files = [f for f in acq.files if f.type in allowed_ftypes]
        
        # Delete info from each file
        for f in files:
            f.delete_info(fields_for_deletion)
            #print("Removed fields from:\t{}".format(f.name))
            logger.info(f"Removing metadata from {f.name}")

logger.info("Done!")

INFO: Processing Session: 008120 (1/2)
INFO: Removing metadata from bbl1_idemo2_210_8.nii.gz
INFO: Removing metadata from epi_singlerep_advshim_22q_5.nii.gz
INFO: Removing metadata from MPRAGE_TI1100_ipat2_2.nii.gz
INFO: Removing metadata from DTI_2x32_36_11.nii.gz
INFO: Removing metadata from B0map_onesizefitsall_v3_6_e2.nii.gz
INFO: Removing metadata from B0map_onesizefitsall_v3_6_e1.nii.gz
INFO: Removing metadata from ep2d_se_pcasl_PHC_1200ms_3.nii.gz
INFO: Removing metadata from localizer_1_i00003.nii.gz
INFO: Removing metadata from localizer_1_i00002.nii.gz
INFO: Removing metadata from localizer_1_i00001.nii.gz
INFO: Removing metadata from bbl1_jolo1_198_9.nii.gz
INFO: Removing metadata from B0map_onesizefitsall_v3_7_e2_ph.nii.gz
INFO: Removing metadata from B0map_onesizefitsall_v3_7_e1_ph.nii.gz
INFO: Removing metadata from ep2d_se_pcasl_PHC_1200ms_4.nii.gz
INFO: Removing metadata from ep2d_se_pcasl_PHC_1200ms_MoCo_4.nii.gz
INFO: Removing metadata from DTI_2x32_35_10.nii.gz
INFO: